# Overview
Use this notebook to convert OCT 3D volume (stored in .tiff)  to virtual histology.

To get started,
[open this notebook in colab](https://colab.research.google.com/github/WinetraubLab/OCT2Hist-ModelInference/blob/main/run_oct2hist3D.ipynb) and run.


In [1]:
# @title Notebook Inputs { display-mode: "form" }
# @markdown Input Image Path
# Path to the OCT image
oct_input_volume_path = "/content/drive/Shareddrives/Yolab - Current Projects/Aidan/2023 Alopecia Test/AlopeciaTest-Region1.tif" # @param {type:"string"}

# OCT image's pixel size
microns_per_pixel_z = 1
microns_per_pixel_x = 1

# @markdown Cropping Parameters
x0 = 0 # @param {type:"slider", min:0, max:1000, step:10}
z0 = 50 # @param {type:"slider", min:0, max:1000, step:10}

output_tiff_path = "/content/drive/Shareddrives/Yolab - Current Projects/Yonatan/Output/" # @param {type:"string"}

min_signal_threshold = 0.46

## Set Up Environment

In [2]:
if not 'setup_env_oct2Hist_usemodel' in globals():
  setup_env_oct2Hist_usemodel = True
  !git clone --recurse-submodules https://github.com/WinetraubLab/OCT2Hist-ModelInference
  !pip install imageio
  %cd OCT2Hist-ModelInference

import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import oct2hist
from utils.show_images import *
from google.colab import drive
from utils.crop import crop
drive.mount('/content/drive/')

Cloning into 'OCT2Hist-ModelInference'...
remote: Enumerating objects: 642, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 642 (delta 77), reused 1 (delta 0), pack-reused 504
Receiving objects: 100% (642/642), 21.54 MiB | 16.40 MiB/s, done.
Resolving deltas: 100% (334/334), done.
Submodule 'pytorch-CycleGAN-and-pix2pix' (https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) registered for path 'pytorch-CycleGAN-and-pix2pix'
Cloning into '/content/OCT2Hist-ModelInference/pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2513, done.        
remote: Total 2513 (delta 0), reused 0 (delta 0), pack-reused 2513        
Receiving objects: 100% (2513/2513), 8.20 MiB | 11.13 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.
Submodule path 'pytorch-CycleGAN-and-pix2pix': checked out '9f8f61e5a375c2e01c5187d093ce9c2409f409b0'
/content/OCT2Hist-ModelInference
Mounted at /content/drive/


In [3]:
# Set up the network environment
oct2hist.setup_network()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Run
In this step we load the OCT image, then crops it to the right size



In [4]:
# Load OCT image
_, oct_volume = cv2.imreadmulti(oct_input_volume_path)
oct_volume = np.asarray(oct_volume)

# Loop over entire volume convert and append to outputfile
for i in range(oct_volume.shape[0]):
  filepath = output_tiff_path + "{0:04}.png".format(i)
  print(filepath)

  # Crop
  oct_image = cv2.cvtColor(oct_volume[i,:,:], cv2.COLOR_GRAY2BGR)
  cropped = crop(oct_image, target_width=1024, target_height=512, x0=x0, z0=z0)

  # Run Inference
  virtual_histology_image, *_ = oct2hist.run_network(
    cropped,
    microns_per_pixel_x = microns_per_pixel_x,
    microns_per_pixel_z = microns_per_pixel_z,
    min_signal_threshold = min_signal_threshold
    )

  # Save to file
  filepath = output_tiff_path + "{0:04}.png".format(i)
  bgr_image = cv2.cvtColor(virtual_histology_image, cv2.COLOR_RGB2BGR)
  cv2.imwrite(filepath, bgr_image)

/content/drive/Shareddrives/Yolab - Current Projects/Yonatan/Output/0000.png
python /content/OCT2Hist-ModelInference/pytorch-CycleGAN-and-pix2pix//test.py --netG resnet_9blocks --dataroot "/content/OCT2Hist-ModelInference/pytorch-CycleGAN-and-pix2pix//dataset/"  --model pix2pix --name oct2hist --checkpoints_dir "/content/OCT2Hist-ModelInference/pytorch-CycleGAN-and-pix2pix//checkpoints" --results_dir "/content/OCT2Hist-ModelInference/pytorch-CycleGAN-and-pix2pix//results" --num_test 1000
Command failed with exit code 1.
Error message:
Traceback (most recent call last):
  File "/content/OCT2Hist-ModelInference/pytorch-CycleGAN-and-pix2pix//test.py", line 43, in <module>
    opt = TestOptions().parse()  # get test options
  File "/content/OCT2Hist-ModelInference/pytorch-CycleGAN-and-pix2pix/options/base_options.py", line 136, in parse
    torch.cuda.set_device(opt.gpu_ids[0])
  File "/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py", line 350, in set_device
    torch._C._cu

RuntimeError: ignored